In [1]:
import underworld as uw

from underworld import function as fn
from underworld.scaling import units as u
from underworld.scaling import dimensionalise as dm, non_dimensionalise as nd

import glucifer

import numpy as np


/Users/arijit/underworld2/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
modelHeight = 2880.0 * u.kilometer
# plateHeight = 120. * u.kilometer
refDensity = 3200.0 * u.kilogram / u.meter ** 3
deltaRhoMax = 80.0 * u.kilogram / u.meter ** 3
gravity = 9.8 * u.metre / u.second ** 2
# 1.57e20 * u.pascal * u.second 5.e20 * u.pascal * u.second
refViscosity = 5.0e20 * u.pascal * u.second
bodyForce = deltaRhoMax * gravity

# scaling coefficients
K_eta = refViscosity
KL = modelHeight
K_tau = bodyForce * modelHeight
K_v = K_tau * modelHeight / K_eta
# Kt = KL/K_v
Kt = K_eta / K_tau
KM = K_tau * modelHeight * Kt ** 2

scaling_coefficients = uw.scaling.get_coefficients()

scaling_coefficients["[length]"] = KL.to_base_units()
# scaling_coefficients["[temperature]"] = KT.to_base_units()
scaling_coefficients["[time]"] = Kt.to_base_units()
scaling_coefficients["[mass]"] = KM.to_base_units()


In [12]:
vRes = 64
resMult = 4  # 64 being the base vRes
aRatioMesh = 2  # xRes/yRes
aRatioCoor = 4  # Model len ratio
yRes = int(vRes * resMult)
xRes = int(vRes * aRatioMesh * resMult)
refineHoriz = True
refineVert = True
# refineHoriz = False
# refineVert = False
refInt = [0.00175, 0.00175]
refRange = [0.5, -0.25]

time = nd(0 * u.megayear)
dt = 0.0
# CFL = 0.1*refInt[1]*yRes
CFL = 1.0


In [13]:
mesh = uw.mesh.FeMesh_Cartesian(
    elementType=("Q1/dQ0"),
    elementRes=(xRes, yRes),
    # minCoord=(nd(0.*u.kilometer), nd(-modelHeight+192.*u.kilometer)),
    minCoord=(nd(0.0 * u.kilometer), nd(-modelHeight)),
    # maxCoord=(nd(9600.*u.kilometer), nd(192.*u.kilometer)),
    maxCoord=(aRatioCoor * nd(modelHeight), nd(0.0 * u.kilometer)),
    periodic=[False, False],
)

bBox = ((mesh.minCoord[0], mesh.minCoord[1]), (mesh.maxCoord[0], mesh.maxCoord[1]))
figSize = (1600 * 2, int(1600 / aRatioCoor) * 2 + 110)

In [14]:
velocityField = mesh.add_variable(nodeDofCount=mesh.dim)
pressureField = mesh.subMesh.add_variable(nodeDofCount=1)
velocityField.data[:] = [0.0, 0.0]
pressureField.data[:] = 0.0

In [15]:
swarm = uw.swarm.Swarm(mesh=mesh)
materialVariable = swarm.add_variable(dataType="int", count=1)
swarmLayout = uw.swarm.layouts.PerCellSpaceFillerLayout(
    swarm=swarm, particlesPerCell=20
)
swarm.populate_using_layout(layout=swarmLayout)
swarm_popcontrol = uw.swarm.PopulationControl(
    swarm,
    deleteThreshold=0.0025,
    splitThreshold=0.10,
    maxDeletions=0,
    maxSplits=100,
    aggressive=True,
    aggressiveThreshold=0.95,
    particlesPerCell=20,
)

In [135]:
def make_slab2d(topX, topY, length, taper, dip, depth, thicknessArray):
    d = topY
    ts = thicknessArray
    noSlabs = len(ts)
    totalThickness = sum(ts)
    polyshapes = []
    for i in range(noSlabs):
        polyshapes.append(
            [
                (topX, topY),
                (topX + totalThickness / (np.tan(np.radians(taper))), d),
                (topX + length, d),
                (topX + length + depth / np.tan(np.radians(dip)), d - depth),
                (topX + length + depth / np.tan(np.radians(dip)), d - depth - ts[i]),
                (topX + length, d - ts[i]),
                (topX + totalThickness / np.tan(np.radians(taper)), d - ts[i]),
            ]
        )
        d -= ts[i]

    return polyshapes


def make_Indentor2d(startX, topY, length, taper, thicknessArray, taper2=None):
    if taper2 is None:
        taper2 = taper
    d = topY
    ts = thicknessArray
    noSlabs = len(ts)
    polyshapes = []
    # xs = ts[0]/np.tan(np.radians(taper))
    for i in range(noSlabs):
        polyshapes.append(
            [
                [startX - d / np.tan(np.radians(taper2)), topY + d],
                [startX + length + d / np.tan(np.radians(taper)), topY + d],
                [
                    startX + length + (d - ts[i]) / np.tan(np.radians(taper)),
                    topY + d - ts[i],
                ],
                [startX - (d - ts[i]) / np.tan(np.radians(taper2)), topY + d - ts[i]],
            ]
        )
        d -= ts[i]

    # polyshapes[0][0][0] = polyshapes[0][0][0]+2*xs
    # polyshapes[0][1][0] = polyshapes[0][1][0]-2*xs
    return polyshapes


def make_overRidingPlate2d(topX, topY, length, taper, dip, thicknessArray):
    d = topY
    ts = thicknessArray
    noPlates = len(ts)
    totalThickness = sum(ts)
    polyshapes = []
    for i in range(noPlates):
        polyshapes.append(
            [
                (topX, topY),
                (topX + totalThickness / np.tan(np.radians(taper)), topY + d),
                (topX + length - d / np.tan(np.radians(dip)), topY + d),
                (
                    topX + length + (ts[i] - d) / np.tan(np.radians(dip)),
                    topY - ts[i] + d,
                ),
                (topX + totalThickness / np.tan(np.radians(taper)), topY - ts[i] + d),
            ]
        )
        d -= ts[i]
    return polyshapes


def make_layer2d(startX, topY, length, thicknessArray):
    d = topY
    ts = thicknessArray
    noSlabs = len(ts)
    polyshapes = []
    for i in range(noSlabs):
        polyshapes.append(
            [
                (startX, d),
                (startX + length, d),
                (startX + length, d - ts[i]),
                (startX, d - ts[i]),
            ]
        )
        d -= ts[i]
    return polyshapes

In [136]:
# Shape Up the Geomotries TODO: Dimentional Coordinates
# mantleandAirShape = make_layer2d(startX=0.,
#                                  topY=mesh.maxCoord[1],
#                                  length=nd(9600.*u.kilometer),
#                                  thicknessArray=[nd(modelHeight)])
# # thicknessArray=[nd(192.*u.kilometer), nd(modelHeight-192.*u.kilometer)])
mantleShape = make_layer2d(
    startX=0.0,
    topY=0.0,
    length=nd(modelHeight * aRatioCoor),
    thicknessArray=[nd(modelHeight)],
    # thicknessArray=[nd(660.*u.kilometer), nd(modelHeight-660.*u.kilometer)]
)

slabshapes = make_slab2d(
    topX=nd(0.725 * modelHeight),
    topY=0.0,
    length=nd(1.275 * modelHeight),
    taper=15,
    dip=29,
    depth=nd(120.0 * u.kilometer),
    thicknessArray=[
        nd(15.0 * u.kilometer),
        nd(15.0 * u.kilometer),
        nd(30.0 * u.kilometer),
        nd(30.0 * u.kilometer),
    ],  # thic # 10 20 20 30
)

indentorshapes = make_Indentor2d(
    startX=nd(0.3 * modelHeight),
    topY=nd(0.0 * u.kilometer),
    length=nd(0.85 * modelHeight),
    taper=18,
    thicknessArray=[
        nd(15.0 * u.kilometer),
        nd(15.0 * u.kilometer),
        nd(30.0 * u.kilometer),
        nd(30.0 * u.kilometer),
    ],  # UL
    taper2=12,
)

overRidingShapesForeArc = make_overRidingPlate2d(
    topX=mesh.maxCoord[0] - mesh.minCoord[0],
    topY=nd(0.0 * u.kilometer),
    length=-nd(2.0 * modelHeight),
    taper=90,
    dip=29,
    thicknessArray=[nd(40.0 * u.kilometer), nd(80.0 * u.kilometer)],
)

overRidingShapes = make_overRidingPlate2d(
    topX=mesh.maxCoord[0] - mesh.minCoord[0],
    topY=nd(0.0 * u.kilometer),
    length=nd(-1.875 * modelHeight),
    taper=90,
    dip=90,
    thicknessArray=[nd(40.0 * u.kilometer), nd(80.0 * u.kilometer)],
)

In [137]:
# define the viscosity Range
viscRange = [1.0, 1e5]


def viscosity_limit(viscosityFn, viscosityRange=viscRange):
    return fn.misc.max(viscosityRange[0], fn.misc.min(viscosityRange[1], viscosityFn))


# the default strain rate Invariant used for the initial visc
defaultSRInv = nd(1e-18 / u.second)
strainRate = fn.tensor.symmetric(velocityField.fn_gradient)
strainRate_2ndInvariant = fn.tensor.second_invariant(strainRate)


def yield_visc(cohesion, viscosity):
    eII = strainRate_2ndInvariant
    etaVp = fn.exception.SafeMaths(
        fn.misc.min(cohesion / (2.0 * (eII + defaultSRInv)), viscosity)
    )
    return viscosity_limit(etaVp)


# depthViscosityfn = fn.branching.conditional([(fn.coord()[1] > nd(-660.*u.kilometer), 1.), (True, 1e2)])
def depthViscosityfn(viscosity0, viscosity1, depth, coordtype=None):
    if isinstance(viscosity0, u.Quantity):
        viscosity0 = nd(viscosity0)
    if isinstance(viscosity1, u.Quantity):
        viscosity1 = nd(viscosity1)

    return fn.branching.conditional(
        [(fn.coord()[1] > nd(depth), viscosity0), (True, viscosity1)]
    )

In [138]:
import collections
modelMaterials=collections
modelMaterials = [
    # {"name": "Air",
    #  "shape": mantleandAirShape[0],
    #  "viscosity":0.1*refViscosity,
    #  "density":0.*u.kilogram / u.meter**3},
    {
        "name": "Mantle",
        "shape": mantleShape[0],
        "viscosity": "deptDependent",
        "eta0": refViscosity,
        "eta1": 1e2 * refViscosity,
        "etaChangeDepth": 660.0 * u.kilometer,
        "density": "deptDependent",
        "rho0": 3200.0 * u.kilogram / u.meter ** 3,
        "rho1": 3230.0 * u.kilogram / u.meter ** 3,
        "rhoChangeDepth": 660.0 * u.kilometer,
    },
    # {"name": 'Upper Mantle',
    #  "shape": mantleShape[0],
    #  "viscosity":1*refViscosity,
    #  "density":3200.*u.kilogram / u.meter**3},
    # {"name": 'Lower Mantle',
    #  "shape": mantleShape[1],
    #  "viscosity":1e2*refViscosity,
    #  "density":3240.*u.kilogram / u.meter**3},
    # Indo-Australian Plate `4 plates`
    {
        "name": "Uppper Crust Indo-Australian Plate",
        "shape": slabshapes[0],
        "viscosity": 1e2 * refViscosity,
        "cohesion": 06.0 * u.megapascal,
        # "viscosity":"deptDependent",
        # "eta0":yield_visc(nd(06.*u.megapascal), nd(1e2*refViscosity)),
        # "eta1":yield_visc(nd(30.*u.megapascal), nd(5e1*refViscosity)),  # 5e1
        # "etaChangeDepth":150.*u.kilometer,
        "density": 3280.0 * u.kilogram / u.meter ** 3,
    },
    {
        "name": "Lower Crust Indo-Australian Plate",
        "shape": slabshapes[1],
        "viscosity": 1e3 * refViscosity,
        "cohesion": 30.0 * u.megapascal,
        "density": 3280.0 * u.kilogram / u.meter ** 3,
    },  # 5.*u.megapascal,
    {
        "name": "Lithospheric Mantle Crust Indo-Australian Plate",
        "shape": slabshapes[2],
        "viscosity": 1e5 * refViscosity,
        "cohesion": 400.0 * u.megapascal,  # hs
        "density": 3280.0 * u.kilogram / u.meter ** 3,
    },
    {
        "name": "Lithospheric Mantle Indo-Australian Plate",
        "shape": slabshapes[3],
        "viscosity": 5e2 * refViscosity,
        "density": 3280.0 * u.kilogram / u.meter ** 3,
        "cohesion": 30.0 * u.megapascal,
    },
    # Indian Indentor
    {
        "name": "Upper Crust Indian Indentor",
        "shape": indentorshapes[0],
        # "viscosity":"deptDependent",
        # "eta0":yield_visc(nd(06.*u.megapascal), nd(1e2*refViscosity)),  # 1e2
        # "eta1":yield_visc(nd(30.*u.megapascal), nd(5e1*refViscosity)),  # 5e1
        # "etaChangeDepth":150.*u.kilometer,
        "viscosity": 1e2 * refViscosity,
        "cohesion": 06.0 * u.megapascal,
        "density": 2800.0 * u.kilogram / u.meter ** 3,
        # "density":"deptDependent",
        # "rho0":2800.*u.kilogram / u.meter**3,
        # "rho1":3280.*u.kilogram / u.meter**3,
        # "rhoChangeDepth":150.*u.kilometer
    },
    {
        "name": "Lower Crust Indian Indentor",
        "shape": indentorshapes[1],
        "viscosity": 1e2 * refViscosity,
        "cohesion": 30.0 * u.megapascal,
        "density": 2800.0 * u.kilogram / u.meter ** 3,
        # "density":"deptDependent",
        # "rho0":2800.*u.kilogram / u.meter**3,
        # "rho1":3280.*u.kilogram / u.meter**3,
        # "rhoChangeDepth":150.*u.kilometer
    },
    {
        "name": "Lithospheric Mantle Indian Indentor",
        "shape": indentorshapes[2],
        "viscosity": 1e5 * refViscosity,
        "cohesion": 400.0 * u.megapascal,
        "density": 3200.0 * u.kilogram / u.meter ** 3,
        # "density":"deptDependent",
        # "rho0":3200.*u.kilogram / u.meter**3,
        # "rho1":3280.*u.kilogram / u.meter**3,
        # "rhoChangeDepth":150.*u.kilometer
    },
    {
        "name": "Lithospheric Mantle Indian Indentor",
        "shape": indentorshapes[3],
        "viscosity": 5e4 * refViscosity,
        "cohesion": 30.0 * u.megapascal,
        "density": 3220.0 * u.kilogram / u.meter ** 3
        # "density":"deptDependent",
        # "rho0":3220.*u.kilogram / u.meter**3,
        # "rho1":3280.*u.kilogram / u.meter**3,
        # "rhoChangeDepth":150.*u.
    },
    # Eurasian Plate
    {
        "name": "Crust Eurasian Plate ForeArc",
        "shape": overRidingShapesForeArc[0],
        "viscosity": 1e3 * refViscosity,
        "density": 3200.0 * u.kilogram / u.meter ** 3,
    },
    {
        "name": "Lithospheric Mantle Eurasian Plate ForeArc",
        "shape": overRidingShapesForeArc[1],
        "viscosity": 5e2 * refViscosity,
        "density": 3200.0 * u.kilogram / u.meter ** 3,
    },
    {
        "name": "Crust Eurasian Plate",
        "shape": overRidingShapes[0],
        "viscosity": 5e2 * refViscosity,
        "density": 3200.0 * u.kilogram / u.meter ** 3,
    },
    {
        "name": "Lithospheric Mantle Eurasian Plate",
        "shape": overRidingShapes[1],
        "viscosity": 2e2 * refViscosity,
        "density": 3200.0 * u.kilogram / u.meter ** 3,
    },
]


In [139]:
import json 
import collections

def dumper(obj):
    try:
        return obj.toJSON()
    except:
        return str(obj)



In [140]:
class QuanityEncoder(json.JSONEncoder):
     def default(self, obj):
         if isinstance(obj, u.Quantity):
             return str(obj)
         return json.JSONEncoder.default(self, obj)


In [141]:

print json.dumps(modelMaterials, indent=1,sort_keys=True,cls=QuanityEncoder,separators=(',', ':')) 


[
 {
  "density":"deptDependent",
  "eta0":"5e+20 pascal * second",
  "eta1":"5e+22 pascal * second",
  "etaChangeDepth":"660.0 kilometer",
  "name":"Mantle",
  "rho0":"3200.0 kilogram / meter ** 3",
  "rho1":"3230.0 kilogram / meter ** 3",
  "rhoChangeDepth":"660.0 kilometer",
  "shape":[
   [
    0.0,
    0.0
   ],
   [
    4.0,
    0.0
   ],
   [
    4.0,
    -1.0
   ],
   [
    0.0,
    -1.0
   ]
  ],
  "viscosity":"deptDependent"
 },
 {
  "cohesion":"6.0 megapascal",
  "density":"3280.0 kilogram / meter ** 3",
  "name":"Uppper Crust Indo-Australian Plate",
  "shape":[
   [
    0.7250000000000001,
    0.0
   ],
   [
    0.8416265877365275,
    0.0
   ],
   [
    2.0,
    0.0
   ],
   [
    2.0751686564696428,
    -0.04166666666666667
   ],
   [
    2.0751686564696428,
    -0.04687500000000001
   ],
   [
    2.0,
    -0.005208333333333334
   ],
   [
    0.8416265877365275,
    -0.005208333333333334
   ]
  ],
  "viscosity":"5e+22 pascal * second"
 },
 {
  "cohesion":"30.0 megapascal"

In [160]:
for i,mat in enumerate(modelMaterials):
    print json.dumps( collections.OrderedDict(mat) , indent=1,cls=QuanityEncoder)



{
 "etaChangeDepth": "660.0 kilometer", 
 "shape": [
  [
   0.0, 
   0.0
  ], 
  [
   4.0, 
   0.0
  ], 
  [
   4.0, 
   -1.0
  ], 
  [
   0.0, 
   -1.0
  ]
 ], 
 "eta0": "5e+20 pascal * second", 
 "name": "Mantle", 
 "density": "deptDependent", 
 "rho1": "3230.0 kilogram / meter ** 3", 
 "rho0": "3200.0 kilogram / meter ** 3", 
 "viscosity": "deptDependent", 
 "rhoChangeDepth": "660.0 kilometer", 
 "eta1": "5e+22 pascal * second"
}
{
 "cohesion": "6.0 megapascal", 
 "shape": [
  [
   0.7250000000000001, 
   0.0
  ], 
  [
   0.8416265877365275, 
   0.0
  ], 
  [
   2.0, 
   0.0
  ], 
  [
   2.0751686564696428, 
   -0.04166666666666667
  ], 
  [
   2.0751686564696428, 
   -0.04687500000000001
  ], 
  [
   2.0, 
   -0.005208333333333334
  ], 
  [
   0.8416265877365275, 
   -0.005208333333333334
  ]
 ], 
 "viscosity": "5e+22 pascal * second", 
 "name": "Uppper Crust Indo-Australian Plate", 
 "density": "3280.0 kilogram / meter ** 3"
}
{
 "cohesion": "30.0 megapascal", 
 "shape": [
  [
   

In [156]:
mMat[0]

OrderedDict([('etaChangeDepth', <Quantity(660.0, 'kilometer')>),
             ('shape', [(0.0, 0.0), (4.0, 0.0), (4.0, -1.0), (0.0, -1.0)]),
             ('eta0', <Quantity(5e+20, 'pascal * second')>),
             ('name', 'Mantle'),
             ('density', 'deptDependent'),
             ('rho1', <Quantity(3230.0, 'kilogram / meter ** 3')>),
             ('rho0', <Quantity(3200.0, 'kilogram / meter ** 3')>),
             ('viscosity', 'deptDependent'),
             ('rhoChangeDepth', <Quantity(660.0, 'kilometer')>),
             ('eta1', <Quantity(5e+22, 'pascal * second')>)])

In [115]:
type(modelMaterials[0]['rho0'])

pint.quantity.Quantity

pint.quantity.Quantity